# K리그 패스 도착점 예측 - 전처리 & 피처 엔지니어링

## 개요
EDA에서 발견한 인사이트를 바탕으로 피처를 생성하고 모델링을 위한 데이터를 준비합니다.

## 목차
1. 데이터 로드
2. 기본 피처 생성
3. 시퀀스 피처 생성
4. Wide Format 변환
5. 피처 선택
6. Y축 대칭 증강

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# 상수 정의
K = 6  # 마지막 K개 이벤트 사용
FIELD_LENGTH = 105
FIELD_WIDTH = 68

## 1. 데이터 로드

In [4]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test_index = pd.read_csv('../data/test.csv')
sample_sub = pd.read_csv('../data/sample_submission.csv')

# 테스트 이벤트 로드
test_events_list = []
for _, row in test_index.iterrows():
    path = row['path'].replace('./', '../data/')
    df_ep = pd.read_csv(path)
    df_ep['game_episode'] = row['game_episode']
    test_events_list.append(df_ep)
test_events = pd.concat(test_events_list, ignore_index=True)

print(f'Train events: {len(train):,}')
print(f'Test events: {len(test_events):,}')

Train events: 356,721
Test events: 53,110


In [5]:
# Train/Test 합치기
train['is_train'] = 1
test_events['is_train'] = 0
events = pd.concat([train, test_events], ignore_index=True)
events['is_home'] = events['is_home'].astype(int)
events = events.sort_values(['game_episode', 'time_seconds', 'action_id']).reset_index(drop=True)

print(f'Total events: {len(events):,}')

Total events: 409,831


## 2. 기본 피처 생성

In [6]:
# 에피소드 내 인덱스
events['event_idx'] = events.groupby('game_episode').cumcount()
events['n_events'] = events.groupby('game_episode')['event_idx'].transform('max') + 1
events['ep_idx_norm'] = events['event_idx'] / (events['n_events'] - 1).clip(lower=1)

print('에피소드 인덱스 피처 생성 완료')

에피소드 인덱스 피처 생성 완료


In [7]:
# 시간 피처
events['prev_time'] = events.groupby('game_episode')['time_seconds'].shift(1)
events['dt'] = (events['time_seconds'] - events['prev_time']).fillna(0.0)
events['episode_duration'] = events.groupby('game_episode')['time_seconds'].transform('max') - \
                             events.groupby('game_episode')['time_seconds'].transform('min')

print('시간 피처 생성 완료')

시간 피처 생성 완료


In [8]:
# 이동 피처
events['dx'] = events['end_x'] - events['start_x']
events['dy'] = events['end_y'] - events['start_y']
events['dist'] = np.sqrt(events['dx']**2 + events['dy']**2)
events['speed'] = events['dist'] / events['dt'].replace(0, 1e-3)
events['angle'] = np.degrees(np.arctan2(events['dy'], events['dx']))

print('이동 피처 생성 완료')

이동 피처 생성 완료


In [9]:
# 골대 관련 피처
events['to_goal_dist'] = np.sqrt((FIELD_LENGTH - events['start_x'])**2 + (FIELD_WIDTH/2 - events['start_y'])**2)
events['to_goal_angle'] = np.degrees(np.arctan2(FIELD_WIDTH/2 - events['start_y'], FIELD_LENGTH - events['start_x']))

print('골대 관련 피처 생성 완료')

골대 관련 피처 생성 완료


In [10]:
# 위치 구역 피처
events['x_zone'] = (events['start_x'] / (FIELD_LENGTH/7)).astype(int).clip(0, 6)
events['lane'] = pd.cut(events['start_y'], bins=[0, FIELD_WIDTH/3, 2*FIELD_WIDTH/3, FIELD_WIDTH], 
                        labels=[0, 1, 2], include_lowest=True).astype(int)
events['is_attacking_third'] = (events['start_x'] > 70).astype(int)
events['is_middle_third'] = ((events['start_x'] >= 35) & (events['start_x'] <= 70)).astype(int)
events['is_center_lane'] = ((events['start_y'] >= 20) & (events['start_y'] <= 48)).astype(int)

print('위치 구역 피처 생성 완료')

위치 구역 피처 생성 완료


In [11]:
# 이전 이벤트 대비 이동
events['prev_start_x'] = events.groupby('game_episode')['start_x'].shift(1)
events['prev_start_y'] = events.groupby('game_episode')['start_y'].shift(1)
events['start_move_x'] = events['start_x'] - events['prev_start_x']
events['start_move_y'] = events['start_y'] - events['prev_start_y']
events['start_move_dist'] = np.sqrt(events['start_move_x']**2 + events['start_move_y']**2)
events['attacking_progress'] = events['start_move_x']

print('이전 이벤트 대비 이동 피처 생성 완료')

이전 이벤트 대비 이동 피처 생성 완료


## 3. 시퀀스 피처 생성

In [12]:
# 메타 정보 추출 (마지막 이벤트 기준)
all_last_events = events.groupby('game_episode', as_index=False).tail(1).copy()
ep_meta_all = all_last_events[['game_episode', 'game_id', 'team_id', 'is_home', 'period_id']].copy()
ep_meta_all = ep_meta_all.rename(columns={'team_id': 'final_team_id'})

# 최종 팀 여부
events = events.merge(ep_meta_all[['game_episode', 'final_team_id']], on='game_episode', how='left')
events['is_final_team'] = (events['team_id'] == events['final_team_id']).astype(int)

print('시퀀스 피처 생성 완료')

시퀀스 피처 생성 완료


In [13]:
# ⚠️ 중요: 타겟 누출 방지
# 마지막 이벤트의 end_x, end_y는 타겟이므로 피처에서 제외
events['last_idx'] = events.groupby('game_episode')['event_idx'].transform('max')
events['is_last'] = (events['event_idx'] == events['last_idx']).astype(int)
mask_last = events['is_last'] == 1

# 마지막 이벤트의 end 관련 피처 제거
for col in ['end_x', 'end_y', 'dx', 'dy', 'dist', 'speed', 'angle']:
    events.loc[mask_last, col] = np.nan

print('타겟 누출 방지 처리 완료')

타겟 누출 방지 처리 완료


In [14]:
# 범주형 변수 인코딩
events['type_name'] = events['type_name'].fillna('__NA__')
events['result_name'] = events['result_name'].fillna('__NA__')

le_type = LabelEncoder()
le_res = LabelEncoder()
events['type_id'] = le_type.fit_transform(events['type_name'])
events['res_id'] = le_res.fit_transform(events['result_name'])
events['team_id_enc'] = events['team_id'].astype(int)

print('범주형 인코딩 완료')
print(f'type_name 클래스: {len(le_type.classes_)}')
print(f'result_name 클래스: {len(le_res.classes_)}')

범주형 인코딩 완료
type_name 클래스: 26
result_name 클래스: 9


## 4. Wide Format 변환

마지막 K개 이벤트를 열(column)로 펼쳐서 하나의 행으로 만듭니다.

In [15]:
# 마지막 K개 이벤트 추출
events['rev_idx'] = events.groupby('game_episode')['event_idx'].transform(lambda s: s.max() - s)
lastK = events[events['rev_idx'] < K].copy()

# K개 이내인 에피소드 처리 (패딩)
def assign_pos_in_K(df):
    df = df.sort_values('event_idx')
    L = len(df)
    df = df.copy()
    df['pos_in_K'] = np.arange(K - L, K)  # 뒤에서부터 채움
    return df

lastK = lastK.groupby('game_episode', group_keys=False).apply(assign_pos_in_K)

print(f'마지막 {K}개 이벤트 추출: {len(lastK):,}')

마지막 6개 이벤트 추출: 100,253


In [16]:
# Wide Format 변환에 사용할 피처
num_cols = ['start_x', 'start_y', 'end_x', 'end_y', 'dx', 'dy', 'dist', 'speed', 'dt', 'angle',
            'to_goal_dist', 'to_goal_angle', 'ep_idx_norm', 'x_zone', 'lane', 'is_final_team',
            'start_move_x', 'start_move_y', 'start_move_dist', 'attacking_progress',
            'is_attacking_third', 'is_middle_third', 'is_center_lane', 'episode_duration']
cat_cols = ['type_id', 'res_id', 'team_id_enc', 'is_home', 'period_id', 'is_last']
feature_cols = [c for c in num_cols + cat_cols if c in lastK.columns]

print(f'피처 수: {len(feature_cols)}')

피처 수: 30


In [17]:
# Pivot으로 Wide Format 변환
wide = lastK[['game_episode', 'pos_in_K'] + feature_cols].copy()
for col in feature_cols:
    wide[col] = pd.to_numeric(wide[col], errors='coerce')

wide_pivot = wide.pivot_table(index='game_episode', columns='pos_in_K', values=feature_cols, aggfunc='first')
wide_pivot.columns = [f'{c}_{int(pos)}' for (c, pos) in wide_pivot.columns]
X = wide_pivot.reset_index()

# 메타 정보 추가
X = X.merge(ep_meta_all[['game_episode', 'game_id', 'final_team_id', 'is_home', 'period_id']], 
            on='game_episode', how='left')

print(f'Wide Format: {X.shape}')

Wide Format: (17849, 178)


In [18]:
# 타겟 추가
train_last = train.groupby('game_episode', as_index=False).tail(1)
labels = train_last[['game_episode', 'end_x', 'end_y']].copy()
labels = labels.rename(columns={'end_x': 'target_x', 'end_y': 'target_y'})

X = X.merge(labels, on='game_episode', how='left')

# Train/Test 분리
train_mask = X['game_episode'].isin(labels['game_episode'])
X_train = X[train_mask].copy()
X_test = X[~train_mask].copy()

print(f'Train: {len(X_train)}, Test: {len(X_test)}')

Train: 15435, Test: 2414


## 5. 피처 선택

In [19]:
import lightgbm as lgb

# 타겟 및 그룹
y_train_x = X_train['target_x'].astype(float).values
y_train_y = X_train['target_y'].astype(float).values
groups = X_train['game_id'].values

# 피처 컬럼
drop_cols = ['game_episode', 'game_id', 'target_x', 'target_y', 'final_team_id', 'is_home', 'period_id']
feature_cols_final = [c for c in X_train.columns if c not in drop_cols]

X_train_feat = X_train[feature_cols_final].fillna(0).astype(float)
X_test_feat = X_test[[c for c in feature_cols_final if c in X_test.columns]].fillna(0).astype(float)

print(f'전체 피처 수: {len(feature_cols_final)}')

전체 피처 수: 173


In [20]:
# LightGBM으로 피처 중요도 계산
lgb_base = {'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.03, 'num_leaves': 127, 'verbose': -1}
model_x = lgb.train(lgb_base, lgb.Dataset(X_train_feat, label=y_train_x), num_boost_round=100)
model_y = lgb.train(lgb_base, lgb.Dataset(X_train_feat, label=y_train_y), num_boost_round=100)

# 평균 중요도
imp_avg = (model_x.feature_importance(importance_type='gain') + model_y.feature_importance(importance_type='gain')) / 2
feat_imp = pd.DataFrame({'feature': X_train_feat.columns, 'importance': imp_avg}).sort_values('importance', ascending=False)

# 상위 8% 선택
n_top = max(int(len(feat_imp) * 0.08), 20)
top_features = feat_imp.head(n_top)['feature'].tolist()

print(f'선택된 피처 수: {len(top_features)}')
print(f'\n상위 10개 피처:')
print(feat_imp.head(10))

선택된 피처 수: 20

상위 10개 피처:
                  feature    importance
160        to_goal_dist_5  4.435557e+07
154       to_goal_angle_5  3.717912e+07
142             start_y_5  7.550799e+06
41                end_y_4  5.978533e+06
21                   dt_5  4.399242e+06
136             start_x_5  3.019380e+06
165             type_id_4  1.828368e+06
36                end_x_4  1.724262e+06
130        start_move_y_5  1.502254e+06
10   attacking_progress_5  1.302148e+06


## 6. Y축 대칭 증강

축구장은 Y축 기준 대칭이므로, Y 관련 피처를 반전시켜 데이터를 2배로 증강합니다.

In [21]:
# Y축 관련 피처 식별
y_cols_in_features = []
for col in top_features:
    col_lower = col.lower()
    if '_y_' in col or col.endswith('_y') or 'start_y' in col or 'end_y' in col:
        y_cols_in_features.append(col)
    elif 'angle' in col_lower or 'lane' in col_lower or 'dy_' in col or 'start_move_y' in col:
        y_cols_in_features.append(col)

print(f'Y축 관련 피처: {len(y_cols_in_features)}개')
print(y_cols_in_features)

Y축 관련 피처: 6개
['to_goal_angle_5', 'start_y_5', 'end_y_4', 'start_move_y_5', 'angle_4', 'dy_4']


In [22]:
def augment_y_axis(df_feat, y_cols):
    """
    Y축 대칭 증강 함수
    - Y 좌표: FIELD_WIDTH - y
    - 각도: -angle
    - dy: -dy
    - lane: 2 - lane
    """
    df_aug = df_feat.copy()
    for col in y_cols:
        if col not in df_aug.columns:
            continue
        col_lower = col.lower()
        if '_y_' in col or col.endswith('_y') or 'start_y' in col or 'end_y' in col:
            df_aug[col] = FIELD_WIDTH - df_aug[col]
        elif 'angle' in col_lower:
            df_aug[col] = -df_aug[col]
        elif 'dy_' in col or 'start_move_y' in col:
            df_aug[col] = -df_aug[col]
        elif 'lane' in col_lower:
            df_aug[col] = 2 - df_aug[col]
    return df_aug

print('Y축 증강 함수 정의 완료')

Y축 증강 함수 정의 완료


In [23]:
# 증강 예시
X_train_df = X_train_feat[top_features].copy()
X_train_aug = augment_y_axis(X_train_df, y_cols_in_features)

print(f'원본: {len(X_train_df)}')
print(f'증강 후: {len(X_train_df) + len(X_train_aug)}')
print('\n증강 전후 비교 (Y 관련 피처):')
if y_cols_in_features:
    col = y_cols_in_features[0]
    print(f'{col}: {X_train_df[col].iloc[0]:.2f} → {X_train_aug[col].iloc[0]:.2f}')

원본: 15435
증강 후: 30870

증강 전후 비교 (Y 관련 피처):
to_goal_angle_5: -78.12 → 78.12


In [24]:
# 최종 데이터 저장
print('=' * 60)
print('전처리 완료 요약')
print('=' * 60)
print(f'''
1. 피처 수: {len(feature_cols_final)} → {len(top_features)} (상위 8%)
2. Train: {len(X_train)}, Test: {len(X_test)}
3. Y축 증강: 2배 증가
4. 타겟 누출: 마지막 이벤트 end 정보 제거됨

주요 피처:
- start_x, start_y (시작 위치)
- to_goal_dist (골대까지 거리)
- dist, angle (이동 거리/각도)
- type_id (이벤트 타입)
''')

전처리 완료 요약

1. 피처 수: 173 → 20 (상위 8%)
2. Train: 15435, Test: 2414
3. Y축 증강: 2배 증가
4. 타겟 누출: 마지막 이벤트 end 정보 제거됨

주요 피처:
- start_x, start_y (시작 위치)
- to_goal_dist (골대까지 거리)
- dist, angle (이동 거리/각도)
- type_id (이벤트 타입)

